In [ ]:
##Longhand - M. Cook - 2022
##Takes text corpus and returns an immersive visualization
##https://github.com/Cook4986/Longhand

In [ ]:
#Takes plain text document and outputs dictionary of common nouns or named entities [key]; relative percentage of occurance, Sketchfab models/uids/URLs [values] 
import spacy
from collections import Counter
import json
import requests
from requests.exceptions import RequestException
import time

#select language models here: https://spacy.io/models
nlp = spacy.load("it_core_news_lg")
nlp = spacy.load("en_core_web_lg")

#I/O
BoW = "...txt"
output = ".../objects.txt"

#declarations
model_size = 100000 #face count
SKETCHFAB_API_URL = "https://api.sketchfab.com/v3/search?type=models&count=1&max_face_count=" + str(model_size)
API_TOKEN = '...'
results = 100 #desired number of objects in visualization
start = time.time()

#data structures
nouns = [] #all nouns in Bag-of-Words
entities = [] #named entities
freqs = [] # noun appearance frequencies
objects = {} # key = common nouns; value(s) = [relative percentage of total objects, UID, model name, URL]
    
#parse Bag-of-Words parts-of-speech with SpaCy
with open(BoW, encoding="utf-8") as file:
    iliad = file.read()
document = nlp(iliad)
document.text

#collate nouns in corpus
for token in document:
    if token.pos_ == 'NOUN': 
        nouns.append(token.text)

#collate named entities, limited to categories with presumed sketchfab correspondences
labelsEN = ["PRODUCT","EVENT","FAC","WORK_OF_ART","LOC","NORP","GPE","ORG"] #english model, add "PERSON"
labelsIT = ["LOC", "MISC", "ORG",] #italian model, add "PER"
for entity in document.ents:
    for label in labelsIT:
        if entity.label_ == label:
            entities.append(entity.text)
file.close()
print("nlp step complete")
print("\n")

#frequency & dictionary creation with common words
word_freq = Counter(nouns) #nouns or entities
common = word_freq.most_common(results)
for word in common:
    freqs.append(word[1])
Sum = sum(freqs)
for word in common:
    flowt = (word[1] / Sum) * 100
    percentage = round(flowt, 2)
    objects[word[0]] = [percentage]
print("object dictionary created")
print("\n")

#Sketchfab API payload function 
##From https://sketchfab.com/developers/data-api/v3/python#example-python-model
def _get_request_payload(*, data=None, files=None, json_payload=False):
    """Helper method that returns the authentication token and proper content type depending on
    whether or not we use JSON payload."""
    data = data or {}
    files = files or {}
    headers = {'Authorization': 'Token {}'.format(API_TOKEN)}
    if json_payload:
        headers.update({'Content-Type': 'application/json'})
        data = json.dumps(data)
    return {'data': data, 'files': files, 'headers': headers}

#query sketchfabs with nouns in BoW and return/write list of uids + model names
for key in objects.keys():
    print("Searching: " + str(key))
    print("\n")
    query = ("&q="+(str(key))+"&downloadable=true")
    search_endpoint = f'{SKETCHFAB_API_URL + query}'
    payload = _get_request_payload() 
    response = requests.get(search_endpoint, **payload)
    data = response.json()
    for item in range(len(data['results'])):
        #if (str((data['results'][item]['name']))) != 0:
        url = (data['results'][item]['uri'])
        uid = (data['results'][item]['uid'])
        name = (str((data['results'][item]['name'])))
        objects[key] += [name, uid, url]
        print("the following model been located: ")
        print(str((data['results'][item]['name']))+(" \nuid: ")+(data['results'][item]['uid']))
        print("\n")

#write to disk and close program
with open(output, 'w') as file:
    file.write(json.dumps(objects)) 
file.close()
print(str(results) + " objects written to disk")
print("\n")
end = time.time()
print(str(end - start) + " seconds elapsed" )
print("\n")
print("have a nice day")


In [ ]:
##Launches Blender from terminal and initiates model download script
!.../Blender --python .../Longhand_downloader.py 
